<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Proximal_Policy_Optimization_(PPO).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade gym stable-baselines3

In [ ]:
pip install 'shimmy>=2.0'

In [ ]:
pip install gymnasium

In [ ]:
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# Create the environment using Gym
env = gym.make('CartPole-v1')

# Optionally wrap the environment with step_api_compatibility for newer Gym versions
env = gym.wrappers.StepAPICompatibility(env)

# Wrap the environment with DummyVecEnv for compatibility with Stable Baselines3
env = DummyVecEnv([lambda: env])

# Initialize the PPO model
model = PPO('MlpPolicy', env, verbose=1)

# Train the model
model.learn(total_timesteps=10000)

# Evaluate the trained model
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs)
    output = env.step(action)  # Adjusted to handle latest Gym API

    if len(output) == 5:
        obs, rewards, dones, truncated, infos = output
    else:
        obs, rewards, dones, infos = output
        truncated = [False] * len(dones)

    env.render()
    # Handle the done flag appropriately
    if dones[0] or truncated[0]:
        obs = env.reset()
env.close()